# example script to try to train and eval BERT model

## 1. eval pretrained

In [28]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
  
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-sk")

model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-sk")

loading configuration file https://huggingface.co/Helsinki-NLP/opus-mt-en-sk/resolve/main/config.json from cache at /Users/marek/.cache/huggingface/transformers/21985b04fd96a9dd3478062ef87ca2d9c76f59a52e4d5e693fad88f4fc595348.6350a3ce87a125fb14fc698c04057fe6248203c7063e3e8d8ea3debdb828a820
Model config MarianConfig {
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "swish",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "MarianMTModel"
  ],
  "attention_dropout": 0.0,
  "bad_words_ids": [
    [
      60024
    ]
  ],
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 512,
  "decoder_attention_heads": 8,
  "decoder_ffn_dim": 2048,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 60024,
  "dropout": 0.1,
  "encoder_attention_heads": 8,
  "encoder_ffn_dim": 2048,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 0,
  "forced_eos_token_id": 

In [29]:
from datasets import load_dataset

dataset = load_dataset("opus100", "en-sk")

Reusing dataset opus100 (/Users/marek/.cache/huggingface/datasets/opus100/en-sk/0.0.0/a87abd612d82947c7a2c3991f71095a98f55141af7ad37516dfb31bfa3511ddc)


In [30]:
tokenizer

PreTrainedTokenizer(name_or_path='Helsinki-NLP/opus-mt-en-sk', vocab_size=60025, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'})

In [31]:
model

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(60025, 512, padding_idx=60024)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(60025, 512, padding_idx=60024)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0): MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
   

In [32]:
dataset

DatasetDict({
    test: Dataset({
        features: ['translation'],
        num_rows: 2000
    })
    train: Dataset({
        features: ['translation'],
        num_rows: 1000000
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 2000
    })
})

In [33]:
dataset['test'][1]

{'translation': {'en': 'What do you mean?', 'sk': 'Čo tým myslíš?'}}

In [34]:
dataset['train'][1]['translation']['en']

'My ankle!'

In [35]:
dataset['train'][1]['translation']['sk']

'Oh, môj členok!'

In [36]:
dataset

DatasetDict({
    test: Dataset({
        features: ['translation'],
        num_rows: 2000
    })
    train: Dataset({
        features: ['translation'],
        num_rows: 1000000
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 2000
    })
})

In [37]:
dataset['train'].features

{'translation': Translation(languages=['en', 'sk'], id=None)}

In [38]:
max_input_length = 512
max_target_length = 512
source_lang = "en"
target_lang = "sk"
prefix = ""

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [39]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Loading cached processed dataset at /Users/marek/.cache/huggingface/datasets/opus100/en-sk/0.0.0/a87abd612d82947c7a2c3991f71095a98f55141af7ad37516dfb31bfa3511ddc/cache-849df37914878351.arrow
Loading cached processed dataset at /Users/marek/.cache/huggingface/datasets/opus100/en-sk/0.0.0/a87abd612d82947c7a2c3991f71095a98f55141af7ad37516dfb31bfa3511ddc/cache-52900ee4dcddce6e.arrow
Loading cached processed dataset at /Users/marek/.cache/huggingface/datasets/opus100/en-sk/0.0.0/a87abd612d82947c7a2c3991f71095a98f55141af7ad37516dfb31bfa3511ddc/cache-f17a927c7c2d8127.arrow


In [40]:
tokenized_datasets

DatasetDict({
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'translation'],
        num_rows: 2000
    })
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'translation'],
        num_rows: 1000000
    })
    validation: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'translation'],
        num_rows: 2000
    })
})

In [41]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [42]:
from transformers import Seq2SeqTrainingArguments

batch_size = 4
# model_name = model_checkpoint.split("/")[-1]

training_args = Seq2SeqTrainingArguments(
    "Helsinki-NLP/opus-mt-en-sk",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=False,
    push_to_hub=False,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [43]:
from datasets import load_metric

metric = load_metric("sacrebleu")

In [44]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
#     print(preds[0])
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [45]:
val_small = tokenized_datasets['validation'].train_test_split(test_size=0.01)

Loading cached split indices for dataset at /Users/marek/.cache/huggingface/datasets/opus100/en-sk/0.0.0/a87abd612d82947c7a2c3991f71095a98f55141af7ad37516dfb31bfa3511ddc/cache-9da5f1743d9f87a0.arrow and /Users/marek/.cache/huggingface/datasets/opus100/en-sk/0.0.0/a87abd612d82947c7a2c3991f71095a98f55141af7ad37516dfb31bfa3511ddc/cache-13dc137a9fe07d0a.arrow


In [46]:
val_small

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'translation'],
        num_rows: 1980
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'translation'],
        num_rows: 20
    })
})

In [47]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [48]:
val_small['test']

Dataset({
    features: ['attention_mask', 'input_ids', 'labels', 'translation'],
    num_rows: 20
})

In [49]:
trainer.predict(val_small['test'])

The following columns in the test set  don't have a corresponding argument in `MarianMTModel.forward` and have been ignored: translation.
***** Running Prediction *****
  Num examples = 20
  Batch size = 4


PredictionOutput(predictions=array([[60024,  7367, 19824, ..., 60024, 60024, 60024],
       [60024, 25312,    52, ..., 60024, 60024, 60024],
       [60024,  8964,     2, ..., 60024, 60024, 60024],
       ...,
       [60024,  2548,  2600, ..., 60024, 60024, 60024],
       [60024,  5759,  1387, ..., 60024, 60024, 60024],
       [60024,   173,   792, ..., 60024, 60024, 60024]]), label_ids=array([[ 7367, 19824,   461, ..., 60024, 60024, 60024],
       [25312,    24,    64, ..., 60024, 60024, 60024],
       [19004,  1387,     2, ..., 60024, 60024, 60024],
       ...,
       [ 2548,  2600,    24, ..., 60024, 60024, 60024],
       [ 5759,  1387,  6960, ..., 60024, 60024, 60024],
       [  173,   792,   289, ..., 60024, 60024, 60024]]), metrics={'eval_loss': 1.3378320932388306, 'eval_bleu': 45.5679, 'eval_gen_len': 10.3, 'eval_runtime': 5.0867, 'eval_samples_per_second': 3.932, 'eval_steps_per_second': 0.983})

We can see **BLEU score of 34.2418**

Other metrics:

'eval_gen_len': 16.85, 'eval_runtime': 12.4035, 'eval_samples_per_second': 1.612, 'eval_steps_per_second': 0.403

## Quantized model evaluation

In [50]:
import torch

torch.backends.quantized.engine = 'qnnpack'

quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)
quantized_model

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(60025, 512, padding_idx=60024)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(60025, 512, padding_idx=60024)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0): MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): DynamicQuantizedLinear(in_features=512, out_features=512, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (v_proj): DynamicQuantizedLinear(in_features=512, out_features=512, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (q_proj): DynamicQuantizedLinear(in_features=512, out_features=512, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (out_proj): DynamicQuantizedLinear(in_features=512, out_features=512, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (

In [51]:
quant_data_collator = DataCollatorForSeq2Seq(tokenizer, model=quantized_model)

In [52]:
from transformers import Seq2SeqTrainer

quant_trainer = Seq2SeqTrainer(
    quantized_model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=quant_data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [53]:
quant_trainer.predict(val_small['test'])

The following columns in the test set  don't have a corresponding argument in `MarianMTModel.forward` and have been ignored: translation.
***** Running Prediction *****
  Num examples = 20
  Batch size = 4


PredictionOutput(predictions=array([[60024,  7367, 19824, ..., 60024, 60024, 60024],
       [60024, 25312,    52, ..., 60024, 60024, 60024],
       [60024,  8964,     2, ..., 60024, 60024, 60024],
       ...,
       [60024,  2548,  2600, ..., 60024, 60024, 60024],
       [60024,  5759,  1387, ..., 60024, 60024, 60024],
       [60024,   173,   792, ..., 60024, 60024, 60024]]), label_ids=array([[ 7367, 19824,   461, ..., 60024, 60024, 60024],
       [25312,    24,    64, ..., 60024, 60024, 60024],
       [19004,  1387,     2, ..., 60024, 60024, 60024],
       ...,
       [ 2548,  2600,    24, ..., 60024, 60024, 60024],
       [ 5759,  1387,  6960, ..., 60024, 60024, 60024],
       [  173,   792,   289, ..., 60024, 60024, 60024]]), metrics={'eval_loss': 1.3460700511932373, 'eval_bleu': 45.5679, 'eval_gen_len': 10.3, 'eval_runtime': 2.7758, 'eval_samples_per_second': 7.205, 'eval_steps_per_second': 1.801})

We can see **BLEU score of 34.842**

Other metrics:

'eval_gen_len': 16.7, 'eval_runtime': 6.1269, 'eval_samples_per_second': 3.264, 'eval_steps_per_second': 0.816

## Test on whole validation set

In [54]:
trainer.predict(tokenized_datasets["validation"])

The following columns in the test set  don't have a corresponding argument in `MarianMTModel.forward` and have been ignored: translation.
***** Running Prediction *****
  Num examples = 2000
  Batch size = 4


PredictionOutput(predictions=array([[60024,  5275,    24, ..., 60024, 60024, 60024],
       [60024,  1740,    64, ..., 60024, 60024, 60024],
       [60024,  1511,     2, ..., 60024, 60024, 60024],
       ...,
       [60024,  2187,     2, ..., 60024, 60024, 60024],
       [60024,  6211,  1995, ..., 60024, 60024, 60024],
       [60024,  5875,   127, ..., 60024, 60024, 60024]]), label_ids=array([[ 1014,    24,   111, ..., 60024, 60024, 60024],
       [ 1740,    64,   501, ..., 60024, 60024, 60024],
       [ 1511,     2,   187, ..., 60024, 60024, 60024],
       ...,
       [ 2187,     2,    43, ..., 60024, 60024, 60024],
       [ 6211,  1995,   206, ..., 60024, 60024, 60024],
       [ 5875,   127,    34, ..., 60024, 60024, 60024]]), metrics={'eval_loss': 1.437319278717041, 'eval_bleu': 35.7476, 'eval_gen_len': 14.988, 'eval_runtime': 894.6245, 'eval_samples_per_second': 2.236, 'eval_steps_per_second': 0.559})

We can see **BLEU score of 35.7476**

Other metrics:

'eval_gen_len': 14.988, **'eval_runtime': 894.6245, 'eval_samples_per_second': 2.236,** 'eval_steps_per_second': 0.559

In [55]:
quant_trainer.predict(tokenized_datasets["validation"])

The following columns in the test set  don't have a corresponding argument in `MarianMTModel.forward` and have been ignored: translation.
***** Running Prediction *****
  Num examples = 2000
  Batch size = 4


PredictionOutput(predictions=array([[60024,  5275,    24, ..., 60024, 60024, 60024],
       [60024,  1740,    64, ..., 60024, 60024, 60024],
       [60024,  1511,     2, ..., 60024, 60024, 60024],
       ...,
       [60024,  2187,     2, ..., 60024, 60024, 60024],
       [60024,  6211,  1995, ..., 60024, 60024, 60024],
       [60024,  5875,   127, ..., 60024, 60024, 60024]]), label_ids=array([[ 1014,    24,   111, ..., 60024, 60024, 60024],
       [ 1740,    64,   501, ..., 60024, 60024, 60024],
       [ 1511,     2,   187, ..., 60024, 60024, 60024],
       ...,
       [ 2187,     2,    43, ..., 60024, 60024, 60024],
       [ 6211,  1995,   206, ..., 60024, 60024, 60024],
       [ 5875,   127,    34, ..., 60024, 60024, 60024]]), metrics={'eval_loss': 1.4529672861099243, 'eval_bleu': 35.7055, 'eval_gen_len': 15.0445, 'eval_runtime': 501.4448, 'eval_samples_per_second': 3.988, 'eval_steps_per_second': 0.997})

We can see **BLEU score of 35.7055**

Other metrics:

'eval_gen_len': 15.0445, **'eval_runtime': 501.4448, 'eval_samples_per_second': 3.988,** 'eval_steps_per_second': 0.997

In [60]:
quant_runtime= 501.4448
full_runtime= 894.6245
print(f"Quantization evaluation is {quant_runtime/full_runtime}% of full precision time")
print(f"Quantization evaluation is {full_runtime/quant_runtime}x faster than full precision time")

Quantization evaluation is 0.5605086826931299% of full precision time
Quantization evaluation is 1.7840936828939098x faster than full precision time


## different dataset

In [45]:
from datasets import load_dataset

dataset = load_dataset("open_subtitles", lang1="en",lang2="sk")

Using custom data configuration en-sk-lang1=en,lang2=sk
Reusing dataset open_subtitles (/Users/marek/.cache/huggingface/datasets/open_subtitles/en-sk-lang1=en,lang2=sk/0.0.0/c1ec973ca4b6e588740d8f167cc0e24ea3f626e70bc7ffe467e944730500e198)


In [47]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'meta', 'translation'],
        num_rows: 8850871
    })
})

In [48]:
dataset['train']

Dataset({
    features: ['id', 'meta', 'translation'],
    num_rows: 8850871
})

In [50]:
dataset['train'][0]

{'id': '0',
 'meta': {'year': 0,
  'imdbId': 3245996,
  'subtitleId': {'en': 5698460, 'sk': 5232687},
  'sentenceIds': {'en': [1], 'sk': [1]}},
 'translation': {'en': 'Subtitles by DramaFever',
  'sk': 'Subtitles by DramaFever'}}

In [51]:
dataset['train'][1]

{'id': '1',
 'meta': {'year': 0,
  'imdbId': 3245996,
  'subtitleId': {'en': 5698460, 'sk': 5232687},
  'sentenceIds': {'en': [2], 'sk': [2]}},
 'translation': {'en': 'Episode 2.', 'sk': 'Epizóda 2'}}